# Regresión Logística: Detección de SPAM

En este ejercicio se muestran los fundamentos de la regresión Logística, planteando uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de Machine Learning: La detección de SPAM.

## Enunciado del Ejercicio

Se propone la construcción de un sistema de aprendizaje automático capaz de predecir si un correo determinado se corresponde con un correo de SPAM o no, para ello, se utilizará el siguinete conjunto de Datos:

##### [2007 TREC Public Spam Corpus](https://terabox.com/s/1cW1v51vsnazpI2PjXAXvCA)
The corpus trec07p contains 75,419 messages:

    25,220 ham
    50,199 spam
    
These messages constitute all the messages delivered to a particular server bteween these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 jul 2007 07:04:53 -0400

### 1.- Funciones Complementarias

En este caso práctico relacionado con la detección de correos electrónicos de SPAM, el conjunto de datos del que se dispone, esta formado por correos electronicos, con sus correspondientes encabezados y campos adicionales. Por lo tanto, se requiere de un procesamiento previo antes de ser ingeridos por el algoritmo de Machine Learning.

In [7]:
# Esta clase facilita el preprocesamineto de correos electrónicos que poseen código HTML.
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []
        
    def handle_data(self, d):
        self.fed.append(d)
        
    def get_data(self):
        return ''.join(self.fed)

In [8]:
# Esta función se encarga de eliminar los tags HTML que se encuentren en el texto del correo.
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [9]:
# Ejemplo de eliminación de los tags HTML de un texto.
t = '<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Además de eliminar los posibles tags HTML que se encuentren en el correo electónico, deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos de correo electrónico que no son relevantes o eliminación de los afijos de una palabra manteniendo únicamente la raiz de la misma (Stemming). La clase que se muestra a continuación realiza estas transformaciones.

In [10]:
import email
import string
import nltk


class Parser:
    
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
        
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
               "body": body,
               "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                           p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions,
        clean the punctuation symbols and stemming of the text."""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Steming of the tokens
        return[self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

##### Lectura de un correo en formato raw

In [11]:
inmail = open("datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

##### Parsing del correo electrónico

In [13]:
p = parser()
p.parser("datasets/trec07p/data/inmail.1")

NameError: name 'parser' is not defined

Lectura de indice

Estas funciones se encargan de cargar en memoria la ruta  de cada uno de  cada uno de los correo selectronicos y su etiqueta correspondiente (spam, ham)

In [ ]:
index = open("datasets/trec07p/data/inmail.1").readlines()
index

In [ ]:
import os

DATASET_PATH = "dataset/trec07"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label": label, "email_path": os.path.join(DATASET_PATH, path)})
    return ret_indexes


In [ ]:
def parse_email(index):
    p = Parser()
    email = p.parse(index["email_Path"])
    return pmail, index["label"]

In [ ]:
indexes = parse_index("datasets/trec07p/full/index", 10)
indexes

## 2.-Preprocesamiento

Con las funciones anteriores se permite la lectura de los correos electronicos de manera programatica y el procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la deteccion de correos de SPAM sin embargo cada unp de lo scorreos sigue estando representado por un diccionario de python con una serie de palabras ok.

In [ ]:
# Cargar el indice y las etiquetas en memoria 
index = parse_index("datasets/trec07p/full/index",1)


In [ ]:
# Leer el primer correo.
import os
open(index[0]["email_path"]).read

In [ ]:
#Parsear el primer correo.
email, label = parse_email(index[0])
print("El correo es:", label)
print (email)